In [1]:
from github import Github
import pickle
import pandas as pd
import pprint
pp = pprint.PrettyPrinter(depth=6)

In [4]:
#using an access token
g = Github("3eccda0bf7e78b97af1eb5b82ede22021331293c")
repo = g.get_repo("SonarSource/sonarqube")

In [5]:
#returned commits in a paginated list
commits = repo.get_commits()

In [26]:
commits_list = []
parents = []
i=0
for commit in commits:
    i+=1
    commits_list.append((commit.sha, commit.commit.message, commit.commit.author.date))
    for parent in commit.commit.parents:
        parents.append((commit.sha, parent.sha))

RateLimitExceededException: 403 {"message": "API rate limit exceeded for user ID 16037020.", "documentation_url": "https://docs.github.com/rest/overview/resources-in-the-rest-api#rate-limiting"}

In [16]:
len(commits_list)

501

In [18]:
# Create pandas df
parentsDf = pd.DataFrame(parents, columns=["commit_sha", "parent_sha"])
parentsDf

,commit_sha,parent_sha
0,6dc2cc9ff6340e6588b7608ed90354aa6767fcdf,0ecd8610552ccd59049308c1be46b03470c48538
1,0ecd8610552ccd59049308c1be46b03470c48538,bc413ee3db73258a22059816bd4766bcea66a9ca
2,bc413ee3db73258a22059816bd4766bcea66a9ca,27282ea04940d30eb33c20a02989a79d88dec4a0
3,27282ea04940d30eb33c20a02989a79d88dec4a0,626469caaa8b060f79f101cffff214d23c7e611b
4,626469caaa8b060f79f101cffff214d23c7e611b,dae0f392c72132be88af4a73d98875fdf477de50
...,...,...
496,88597f3717a873db1373e4d4b2bcca7f5fd74425,9ea4d1c0a65d82acd4c4d3628a82b6a673fcb0c9
497,9ea4d1c0a65d82acd4c4d3628a82b6a673fcb0c9,74384e67058fcfea16c80839740d28d9083b944a
498,74384e67058fcfea16c80839740d28d9083b944a,7294268682b6d1add416c7de5d41422367fec324
499,7294268682b6d1add416c7de5d41422367fec324,4f5231d18118c6643c19f1d1259f1165b2764774


In [24]:
(parentsDf.groupby("commit_sha").count().parent_sha == 1).sum()

501

In [80]:
# Create pandas df
commitsDf = pd.DataFrame(commits_list, columns=["Sha", "Message", "Date"])
commitsDf.Date = pd.to_datetime(commitsDf.Date)

In [81]:
commitsDf

,Sha,Message,Date
0,4b3d914bf578d1eb88fc9ca79efef00843511865,Upgrade Gradle plugins,2020-09-24 18:16:15
1,ad7d964270d6a6c3da52bf54931ce9fbf50c3076,SONAR-13905 upgrade Hazelcast from 3.12.7 to 3...,2020-09-24 08:38:51
2,552f5bbe1d4eca4783a4da1325b16b0cd4d7783f,SONAR-13905 upgrade Tomcat from 8.5.56 zo 8.5....,2020-09-24 08:13:42
3,9a9609134d2c5136baed9436f9f54eb20821a25e,SONAR-10558 Make it explicit that the project'...,2020-09-28 09:05:10
4,c87841b757f4a849ddae0195d3f0bb98e2d5223f,Fix issue with BBS repos fetch,2020-09-25 12:04:29
...,...,...,...
26995,3f099f35475a66e24290c29004cdc6acef49800d,Improve message when database must be upgraded,2012-03-26 08:57:03
26996,2796325b4b87e9b273f7b4fe0649ae3f6297a51b,SONAR-3354 Display empty table instead of mess...,2012-03-27 08:34:42
26997,c3d0fe433d773d015f2fb99521a82b8ab29c0da6,Fix violations,2012-03-27 07:37:20
26998,2d4f0cd2bb0b768f401ca776eb256d7f31957976,Improve DB readme file.,2012-03-27 07:36:25


In [14]:
pp.pprint(commits[0].commit.parents[0].sha)

'0ecd8610552ccd59049308c1be46b03470c48538'


In [18]:
pp.pprint(commits_list[0].__dict__)

{'_CompletableGithubObject__completed': False,
 '_author': <github.GithubObject._ValuedAttribute object at 0x111c77460>,
 '_comments_url': <github.GithubObject._ValuedAttribute object at 0x111c5b100>,
 '_commit': <github.GithubObject._ValuedAttribute object at 0x111c55580>,
 '_committer': <github.GithubObject._ValuedAttribute object at 0x111c58a90>,
 '_files': NotSet,
 '_headers': {'access-control-allow-origin': '*',
              'access-control-expose-headers': 'ETag, Link, Location, '
                                               'Retry-After, X-GitHub-OTP, '
                                               'X-RateLimit-Limit, '
                                               'X-RateLimit-Remaining, '
                                               'X-RateLimit-Used, '
                                               'X-RateLimit-Reset, '
                                               'X-OAuth-Scopes, '
                                               'X-Accepted-OAuth-Scopes, '
          

In [30]:
# Dump tags objects retrieved from Github APIs

tags = repo.get_tags()
f = open('pkl/tags.pkl', 'wb')
for tag in tags:
    pickle.dump(tag, f)

In [31]:
tmp = []
for tag in tags:
    tmp.append((tag.name, tag.commit.sha, repo.get_commit(sha=tag.commit.sha).commit.author.date))

In [27]:
# Create a file with tags pandas df
tagsDf = pd.DataFrame(tmp, columns=["Tag", "Sha", "Date"])
tagsDf.Date = pd.to_datetime(tagsDf.Date)
#tagsDf.to_csv("csv/tags.csv")

NameError: name 'tmp' is not defined

In [ ]:
tags = pd.read_csv("csv/tags.csv")

In [58]:
import datetime
tagsDf[tagsDf.Date > pd.to_datetime(datetime.date(2015,3,10))]

,Tag,Sha,Date
1,8.4.2.36762,2de1f343e3dbd9e88007b42e475512677c04eb8a,2020-08-27 14:11:31
2,8.4.1.35646,a0d372f704998d38856c26e28662fd7de062f873,2020-07-14 08:13:18
3,8.4.0.35506,74b4533439f380827d68e8c01667eeaecad89fc7,2020-07-03 07:09:27
4,8.3.1.34397,b77ce511a2039330aea8832ad12e86a8cfa9ff7d,2020-05-07 12:09:22
5,8.3.0.34182,9c58e36328ae79ad3c36c13aa253879351ce3613,2020-04-28 13:54:13
...,...,...,...
116,5.1,4aa9af3a6a4362b61db365fba32eb0a55d411e7a,2015-04-02 08:09:45
117,5.1-RC2,dfc9f94b72103ee941cc11da6529623fa91df509,2015-03-23 15:22:47
125,4.5.7,e2afb0bff1b8be759789d2c1bc9348de6f519f83,2016-04-08 13:44:39
126,4.5.6,12d931e5a31e1692a428bbb34ba11cd63aebf998,2015-10-16 09:31:18


Get commits differences between two tags
https://stackoverflow.com/questions/40293833/how-to-get-all-commits-in-a-git-tag-through-github-api